In [17]:
import os, re, io
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, inspect, text

import pymysql

from datetime import datetime

# Obtenir la date et l'heure actuelle
NOW = datetime.now()
print('________________________________________________________________')
print(f'service lancé le {NOW}')

# Charger le fichier .env pour se connecter aux bases de données SUGAR et Misval 
load_dotenv()
SUGAR_URL_CONNECT = os.getenv('SUGAR_URL_CONNECT')

# Créer un moteur de connexion à la base de données MySQL de SUGAR, avec les identifiants et informations récupérés des variables d'environnement
engine_sug = create_engine(SUGAR_URL_CONNECT)

inspector = inspect(engine_sug)

# 1. Liste des tables
tables = inspector.get_table_names()

# Récupération et filtrage des tables contenant "research"
tables = [table for table in inspector.get_table_names() if "researchunitsfrc" in table.lower()]

# Afficher les colonnes pour chaque table
for table in tables:
    print(f"\n📌 Table : {table}")
    print("=" * (10 + len(table)))

    # Récupérer les colonnes
    columns = inspector.get_columns(table)

    # Affichage formaté
    for col in columns:
        col_name = col["name"]
        col_type = col["type"]
        nullable = "NULL" if col.get("nullable", True) else "NOT NULL"
        default = col.get("default", "Aucun")
        print(f"  - {col_name} ({col_type}) | {nullable} | Default: {default}")

    print("-" * (10 + len(table)))

________________________________________________________________
service lancé le 2025-10-21 09:28:56.562866

📌 Table : researchunitsfrc
  - id (CHAR(36)) | NOT NULL | Default: None
  - name (VARCHAR(255)) | NULL | Default: None
  - date_entered (DATETIME) | NULL | Default: None
  - date_modified (DATETIME) | NULL | Default: None
  - modified_user_id (CHAR(36)) | NULL | Default: None
  - created_by (CHAR(36)) | NULL | Default: None
  - description (TEXT) | NULL | Default: None
  - deleted (TINYINT) | NULL | Default: '0'
  - budget (DECIMAL(15, 2)) | NULL | Default: None
  - team_id (CHAR(36)) | NULL | Default: None
  - team_set_id (CHAR(36)) | NULL | Default: None
  - assigned_user_id (CHAR(36)) | NULL | Default: None
  - sync_key (VARCHAR(100)) | NULL | Default: None
  - acl_team_set_id (CHAR(36)) | NULL | Default: None
  - coordinatrice (TINYINT) | NULL | Default: '0'
--------------------------

📌 Table : researchunitsfrc_audit
  - id (CHAR(36)) | NOT NULL | Default: None
  - parent_

In [26]:
def etl_frc():

    # Charger le fichier .env pour récupérer les variables d'environnement
    load_dotenv()
    # Récupérer les informations de connexion aux bases de données depuis les variables d'environnement
    SUGAR_URL_CONNECT = os.getenv('SUGAR_URL_CONNECT')

    # Créer une connexion à la base de données SUGAR
    engine_sug = create_engine(SUGAR_URL_CONNECT)

    try:
        connection_sug = engine_sug.connect()
        print("Connexion réussie à la base de données SUGAR distante.")
    # Gérer les exceptions en cas d'erreur lors de la connexion
    except Exception as e:
        print(f"Erreur lors de la connexion : {e}")

    sql_query = ''' 
            SELECT 
                frc.id, 
                frc.name,
                frc.date_entered,
                frc.description,
                frc.code_projet,
                frc.ligne_comptable_genesys,
                frc.acronyme,
                frc.status,
                frc.etatav,
                frc.date_debut,
                frc.date_fin_initiale,
                frc.programme_cadre,
                frc.call_frc,
                frc.ligne_soumission,
                frc.contract_number,
                frc.new_date_end,
                frc.deadline_montage,
                frc.instrument,
                frc.nb_etape,
                frc.nb_partenaire,
                frc.nb_periode,
                frc.wp_it,
                frc.subvention_inserm_initiale,
                frc.subvention_inserm_actuelle,
                frc.subvention_it_initiale,
                frc.subvention_it_actuelle,
                frc.subvention_totale,
                frc.informations_complementaires,
                frc.team_id,
                frc.team_set_id,
                frc.assigned_user_id,
                frc.delegation_regionale_id,
                frc.organisme_coordinateur_id,
                frc.bailleur_id,
                frc.scientifique_principal_id,
                frc.subvention_partenaires,
                frc.annee_financement,
                frc.ga_number,
                frc.totale_initial,
                ru.name AS ru_name,
                ru.acronyme AS ru_acronyme,
                rufrc.coordinatrice
            FROM frc
            LEFT JOIN researchunitsfrc_frc AS rufrcfrc
                ON frc.id = rufrcfrc.frc_id
            LEFT JOIN researchunitsfrc AS rufrc 
                ON rufrcfrc.researchunitsfrc_id = rufrc.id
            LEFT JOIN researchunitsfrc_researchunits AS rufrcru 
                ON rufrcfrc.researchunitsfrc_id = rufrcru.researchunitsfrc_id
            LEFT JOIN researchunits AS ru 
                ON rufrcru.researchunits_id = ru.id
            WHERE frc.deleted = 0;
    '''


    # Exécuter la requête SQL sur la base de données SUGAR via la connexion active
    result = connection_sug.execute(text(sql_query))
    df_frc = pd.read_sql_query(text(sql_query), connection_sug)
    
    df_frc.coordinatrice = df_frc.coordinatrice.fillna(0).astype(int).astype(bool)

    sql_query = ''' 
	SELECT 
	    opp.id, opp.date_entered, opp.name, opp.campaign_id, opp.lead_source, opp.date_closed, opp.sales_stage, opp.sales_status, 
    	    opp.assigned_user_id, opp.service_start_date, opp.is_escalated, opp.denorm_account_name, 
    	    opp_c.nbdays_open_c, opp_c.modele_opportunity_c, opp_c.other_lead_source_c, opp_c.offre_c, 
    	    opp_c.opp_number_c, opp_c.importance_c, opp_con.contact_id, opp_con.contact_role,
	    rucon.status, ru.acronyme
	FROM opportunities AS opp 
	JOIN opportunities_cstm AS opp_c
	    ON opp.id = opp_c.id_c
	LEFT JOIN opportunities_contacts AS opp_con
	    ON opp.id = opp_con.opportunity_id AND opp_con.deleted = 0
	LEFT JOIN researchunitscontacts_contacts AS rucon_con
	    ON rucon_con.contacts_id = opp_con.contact_id
	LEFT JOIN researchunitscontacts AS rucon
	    ON rucon.id = rucon_con.researchunitscontacts_id
	LEFT JOIN researchunitscontacts_researchunits AS rucon_ru
	    ON rucon_ru.researchunitscontacts_id = rucon_con.researchunitscontacts_id
	LEFT JOIN researchunits AS ru
	    ON ru.id = rucon_ru.researchunits_id
	WHERE opp.deleted = 0
	  AND opp_c.isduplicated_c = 0
	  AND opp_c.modele_opportunity_c = 'frc';
	'''
	# Exécuter la requête SQL sur la base de données SUGAR via la connexion active
    result = connection_sug.execute(text(sql_query))
    df_opp = pd.read_sql_query(text(sql_query), connection_sug)
    df_opp = df_opp[df_opp.status != 'etait_rattache_a']

    return df_frc, df_opp

In [27]:
df_frc, df_opp = etl_frc()

Connexion réussie à la base de données SUGAR distante.


In [28]:
df_frc.code_projet.nunique()

546

In [32]:
df_frc[df_frc.coordinatrice].nunique()


id                              254
name                            243
date_entered                    251
description                     218
code_projet                     253
ligne_comptable_genesys          51
acronyme                        243
status                            4
etatav                            8
date_debut                       54
date_fin_initiale                57
programme_cadre                  26
call_frc                        169
ligne_soumission                187
contract_number                  44
new_date_end                     10
deadline_montage                102
instrument                       50
nb_etape                          4
nb_partenaire                    32
nb_periode                        8
wp_it                            25
subvention_inserm_initiale      167
subvention_inserm_actuelle       76
subvention_it_initiale          102
subvention_it_actuelle           59
subvention_totale                98
informations_complementaires